In [1]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

In [2]:
import io
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import tensorflow_datasets as tfds
from tensorflow import keras
from tensorflow.keras import layers

In [3]:
(ds_train, ds_test), ds_info = tfds.load(
    "cifar10",
    split=["train", "test"],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,)

Dataset cifar10 downloaded and prepared to C:\Users\Abdullah\tensorflow_datasets\cifar10\3.0.2. Subsequent calls will reuse this data.


In [4]:
# normalizing images
def normalize_img(image, label):
    return tf.cast(image, tf.float32) / 255.0, label

In [5]:
# data augmentation
def augment(image, label):
    if tf.random.uniform((), minval=0, maxval=1) < 0.1:
        image = tf.tile(tf.image.rgb_to_grayscale(image), [1, 1, 3])

    image = tf.image.random_brightness(image, max_delta=0.1)
    image = tf.image.random_flip_left_right(image)

    return image, label

In [6]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 32

In [7]:
# Setup for train dataset
ds_train = ds_train.map(normalize_img, num_parallel_calls=AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits["train"].num_examples)
ds_train = ds_train.map(augment)
ds_train = ds_train.batch(BATCH_SIZE)
ds_train = ds_train.prefetch(AUTOTUNE)

# Setup for test Dataset
ds_test = ds_train.map(normalize_img, num_parallel_calls=AUTOTUNE)
ds_test = ds_train.batch(BATCH_SIZE)
ds_test = ds_train.prefetch(AUTOTUNE)

In [8]:
# defining class name to project correct label
class_names = [
    "Airplane",
    "Autmobile",
    "Bird",
    "Cat",
    "Deer",
    "Dog",
    "Frog",
    "Horse",
    "Ship",
    "Truck",
]

In [9]:
# customized layer(user defined layer)
def get_model():
    model = keras.Sequential(
        [
            layers.Input((32, 32, 3)),
            layers.Conv2D(8, 3, padding="same", activation="relu"),
            layers.Conv2D(16, 3, padding="same", activation="relu"),
            layers.MaxPooling2D((2, 2)),
            layers.Flatten(),
            layers.Dense(64, activation="relu"),
            layers.Dropout(0.1),
            layers.Dense(10),
        ]
    )

    return model

In [10]:
model= get_model()

In [11]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

In [1]:
# callback is most simple way to use tensorboard
tensorboard_callback = keras.callbacks.TensorBoard(
    log_dir="tb_callback_dir", histogram_freq=1,)

NameError: name 'keras' is not defined

In [13]:
model.fit(ds_train, epochs=5, verbose=2, validation_data=ds_test,callbacks=[tensorboard_callback])

Epoch 1/5
1563/1563 - 150s - loss: 1.5453 - accuracy: 0.4390 - val_loss: 1.2292 - val_accuracy: 0.5653
Epoch 2/5
1563/1563 - 127s - loss: 1.2112 - accuracy: 0.5706 - val_loss: 1.0765 - val_accuracy: 0.6216
Epoch 3/5
1563/1563 - 127s - loss: 1.1081 - accuracy: 0.6099 - val_loss: 0.9841 - val_accuracy: 0.6592
Epoch 4/5
1563/1563 - 130s - loss: 1.0472 - accuracy: 0.6296 - val_loss: 0.9333 - val_accuracy: 0.6733
Epoch 5/5
1563/1563 - 136s - loss: 1.0101 - accuracy: 0.6449 - val_loss: 0.9134 - val_accuracy: 0.6774
